# Experiment 004: Character-Level WaveNet Model

**Objective**: Implement character-level WaveNet refinement to improve span boundary detection

**Expected gain**: +0.015 to +0.025 points
**Priority**: HIGHEST (addresses core weakness in boundary detection)

**Approach**:
1. Modify RoBERTa inference to save token start/end probability distributions
2. Convert token probabilities to character-level probabilities using offset_mapping
3. Build WaveNet with dilated convolutions for smoothing
4. Train on 5-fold CV using character-level probabilities as features
5. Generate refined predictions with better boundary detection

**Reference**: Winning solution by Theo Viel (dark-of-the-moon) - Character-level WaveNet was key innovation

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import json
import sys
import re

print("Libraries imported successfully")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Define Jaccard similarity metric
def jaccard_similarity(str1, str2):
    """Calculate Jaccard similarity between two strings."""
    if pd.isna(str1) or pd.isna(str2):
        return 0.0
    
    str1, str2 = str(str1), str(str2)
    
    # Tokenize by splitting on whitespace
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    
    # Handle empty sets
    if len(a) == 0 and len(b) == 0:
        return 1.0
    
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

Libraries imported successfully
PyTorch version: 2.2.0+cu118
CUDA available: False


In [3]:
# Load training data
train_path = Path('/home/data/train.csv')
train_df = pd.read_csv(train_path)

print(f"Loaded {len(train_df)} training samples")
print(f"Columns: {list(train_df.columns)}")
print(f"\nSentiment distribution:")
print(train_df['sentiment'].value_counts())

# Show sample
print(f"\nSample row:")
print(train_df.iloc[0])

Loaded 27481 training samples
Columns: ['textID', 'text', 'selected_text', 'sentiment']

Sentiment distribution:
sentiment
neutral     11118
positive     8582
negative     7781
Name: count, dtype: int64

Sample row:
textID                                     cb774db0d1
text              I`d have responded, if I were going
selected_text     I`d have responded, if I were going
sentiment                                     neutral
Name: 0, dtype: object


In [4]:
# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
print(f"Tokenizer loaded: {tokenizer.name_or_path}")

# Test tokenization with offset mapping
text = train_df.iloc[0]['text']
selected_text = train_df.iloc[0]['selected_text']

print(f"\nOriginal text: '{text}'")
print(f"Selected text: '{selected_text}'")

# Tokenize with offset mapping
encoding = tokenizer(
    text,
    return_offsets_mapping=True,
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=128
)

print(f"\nToken IDs shape: {encoding['input_ids'].shape}")
print(f"Offset mapping shape: {encoding['offset_mapping'].shape}")
print(f"\nFirst 10 tokens and their offsets:")
for i in range(min(10, len(encoding['input_ids'][0]))):
    token_id = encoding['input_ids'][0][i].item()
    token = tokenizer.decode([token_id])
    offsets = encoding['offset_mapping'][0][i]
    print(f"  {i}: '{token}' -> offsets: {offsets}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Tokenizer loaded: roberta-base

Original text: ' I`d have responded, if I were going'
Selected text: 'I`d have responded, if I were going'

Token IDs shape: torch.Size([1, 12])
Offset mapping shape: torch.Size([1, 12, 2])

First 10 tokens and their offsets:
  0: '<s>' -> offsets: tensor([0, 0])
  1: ' I' -> offsets: tensor([1, 2])
  2: '`' -> offsets: tensor([2, 3])
  3: 'd' -> offsets: tensor([3, 4])
  4: ' have' -> offsets: tensor([5, 9])
  5: ' responded' -> offsets: tensor([10, 19])
  6: ',' -> offsets: tensor([19, 20])
  7: ' if' -> offsets: tensor([21, 23])
  8: ' I' -> offsets: tensor([24, 25])
  9: ' were' -> offsets: tensor([26, 30])


In [6]:
# Function to extract token start/end probabilities from RoBERTa
@torch.no_grad()
def extract_token_probabilities(model, text, sentiment, tokenizer, device='cpu'):
    """
    Extract token-level start and end probability distributions from RoBERTa.
    Returns probabilities for each token position.
    """
    # Prepare input with sentiment token
    input_text = f"{sentiment} {text}"
    encoding = tokenizer(
        input_text,
        return_offsets_mapping=True,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=128
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    offset_mapping = encoding['offset_mapping'][0]  # Remove batch dim
    
    # Get model outputs
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    start_logits = outputs.start_logits[0]  # [seq_len]
    end_logits = outputs.end_logits[0]      # [seq_len]
    
    # Convert to probabilities
    start_probs = F.softmax(start_logits, dim=0).cpu().numpy()
    end_probs = F.softmax(end_logits, dim=0).cpu().numpy()
    
    return {
        'start_probs': start_probs,  # [seq_len]
        'end_probs': end_probs,      # [seq_len]
        'offset_mapping': offset_mapping.cpu().numpy(),  # [seq_len, 2]
        'input_ids': input_ids.cpu().numpy(),
        'attention_mask': attention_mask.cpu().numpy()
    }

# Test the function
print("Testing token probability extraction...")

# Load a trained RoBERTa model (from exp_002)
model_path = Path('/home/code/experiments/002_roberta_span/fold_0_model.pt')
if model_path.exists():
    model = torch.load(model_path, map_location='cpu')
    model.eval()
    print(f"Loaded model from {model_path}")
else:
    # If no saved model, load pretrained
    print("No saved model found, loading pretrained RoBERTa...")
    model = RobertaForQuestionAnswering.from_pretrained('roberta-base')

model = model.to('cpu')

# Test on a sample
test_text = train_df.iloc[0]['text']
test_sentiment = train_df.iloc[0]['sentiment']

probs = extract_token_probabilities(model, test_text, test_sentiment, tokenizer)
print(f"\nStart probabilities shape: {probs['start_probs'].shape}")
print(f"End probabilities shape: {probs['end_probs'].shape}")
print(f"Offset mapping shape: {probs['offset_mapping'].shape}")

# Show top predictions
start_token = np.argmax(probs['start_probs'])
end_token = np.argmax(probs['end_probs'])
print(f"\nPredicted start token: {start_token} (prob: {probs['start_probs'][start_token]:.4f})")
print(f"Predicted end token: {end_token} (prob: {probs['end_probs'][end_token]:.4f})")

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Testing token probability extraction...
No saved model found, loading pretrained RoBERTa...



Start probabilities shape: (14,)
End probabilities shape: (14,)
Offset mapping shape: (14, 2)

Predicted start token: 13 (prob: 0.0831)
Predicted end token: 12 (prob: 0.0772)


In [7]:
# Function to convert token probabilities to character probabilities
def token_to_char_probabilities(token_probs, offset_mapping, text_length):
    """
    Convert token-level probabilities to character-level probabilities.
    
    Args:
        token_probs: [seq_len] array of token probabilities
        offset_mapping: [seq_len, 2] array of (start_char, end_char) offsets
        text_length: Length of original text
        
    Returns:
        char_probs: [text_length] array of character-level probabilities
    """
    char_probs = np.zeros(text_length)
    char_counts = np.zeros(text_length)
    
    # Aggregate probabilities from tokens to characters
    for token_idx, (char_start, char_end) in enumerate(offset_mapping):
        # Skip special tokens (offset = (0, 0))
        if char_start == 0 and char_end == 0:
            continue
            
        # Ensure offsets are within bounds
        char_start = max(0, min(char_start, text_length - 1))
        char_end = max(0, min(char_end, text_length))
        
        if char_end > char_start:
            # Distribute token probability to characters
            prob_per_char = token_probs[token_idx] / (char_end - char_start)
            char_probs[char_start:char_end] += prob_per_char
            char_counts[char_start:char_end] += 1
    
    # Average probabilities where multiple tokens overlap
    mask = char_counts > 0
    char_probs[mask] /= char_counts[mask]
    
    return char_probs

# Test the conversion function
print("Testing token to character probability conversion...")

text_length = len(test_text)
char_start_probs = token_to_char_probabilities(
    probs['start_probs'], 
    probs['offset_mapping'], 
    text_length
)
char_end_probs = token_to_char_probabilities(
    probs['end_probs'], 
    probs['offset_mapping'], 
    text_length
)

print(f"Character-level start probabilities shape: {char_start_probs.shape}")
print(f"Character-level end probabilities shape: {char_end_probs.shape}")

# Show character probabilities for first 50 characters
print(f"\nFirst 50 characters of text: '{test_text[:50]}'")
print("Character start probabilities (first 20):")
for i in range(min(20, text_length)):
    print(f"  Char {i} ('{test_text[i]}'): {char_start_probs[i]:.4f}")

Testing token to character probability conversion...
Character-level start probabilities shape: (36,)
Character-level end probabilities shape: (36,)

First 50 characters of text: ' I`d have responded, if I were going'
Character start probabilities (first 20):
  Char 0 (' '): 0.0086
  Char 1 ('I'): 0.0086
  Char 2 ('`'): 0.0086
  Char 3 ('d'): 0.0086
  Char 4 (' '): 0.0086
  Char 5 ('h'): 0.0086
  Char 6 ('a'): 0.0086
  Char 7 ('v'): 0.0000
  Char 8 ('e'): 0.0000
  Char 9 (' '): 0.0673
  Char 10 ('r'): 0.0597
  Char 11 ('e'): 0.0646
  Char 12 ('s'): 0.0000
  Char 13 ('p'): 0.0180
  Char 14 ('o'): 0.0180
  Char 15 ('n'): 0.0180
  Char 16 ('d'): 0.0180
  Char 17 ('e'): 0.0000
  Char 18 ('d'): 0.0081
  Char 19 (','): 0.0081


In [8]:
# WaveNet architecture for character-level probability refinement
class CharacterWaveNet(nn.Module):
    def __init__(self, input_channels=2, num_classes=2, num_blocks=4, num_layers=6, 
                 residual_channels=32, gate_channels=32, skip_channels=32):
        """
        WaveNet for character-level span prediction refinement.
        
        Args:
            input_channels: Number of input channels (start_prob, end_prob)
            num_classes: Number of output classes (refined_start, refined_end)
            num_blocks: Number of residual blocks
            num_layers: Number of layers per block (dilated convolutions)
            residual_channels: Channels in residual connections
            gate_channels: Channels in gated activation units
            skip_channels: Channels in skip connections
        """
        super(CharacterWaveNet, self).__init__()
        
        self.num_blocks = num_blocks
        self.num_layers = num_layers
        
        # Input projection
        self.input_projection = nn.Conv1d(input_channels, residual_channels, 1)
        
        # Dilated convolution layers
        self.dilated_convs = nn.ModuleList()
        self.residual_convs = nn.ModuleList()
        self.skip_convs = nn.ModuleList()
        
        # Build dilated convolution blocks
        for b in range(num_blocks):
            for l in range(num_layers):
                # Dilation doubles each layer: 1, 2, 4, 8, 16, 32, ...
                dilation = 2 ** l
                
                # Gated activation unit (dilated convolution)
                self.dilated_convs.append(
                    nn.Conv1d(residual_channels, gate_channels, kernel_size=3, 
                             padding=dilation, dilation=dilation)
                )
                
                # 1x1 conv for residual connection
                self.residual_convs.append(
                    nn.Conv1d(gate_channels // 2, residual_channels, 1)
                )
                
                # 1x1 conv for skip connection
                self.skip_convs.append(
                    nn.Conv1d(gate_channels // 2, skip_channels, 1)
                )
        
        # Output layers
        self.output_conv1 = nn.Conv1d(skip_channels, skip_channels, 1)
        self.output_conv2 = nn.Conv1d(skip_channels, num_classes, 1)
        
        # Activation functions
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        """
        Forward pass.
        
        Args:
            x: Input tensor [batch, input_channels, seq_len]
            
        Returns:
            output: Output tensor [batch, num_classes, seq_len]
        """
        # Input projection
        x = self.input_projection(x)  # [batch, residual_channels, seq_len]
        
        # Skip connections accumulator
        skip_connections = []
        
        # Apply dilated convolutions
        layer_idx = 0
        for b in range(self.num_blocks):
            for l in range(self.num_layers):
                # Gated activation: tanh(x) * sigmoid(x)
                dilated_out = self.dilated_convs[layer_idx](x)
                
                # Split into two parts for gating
                tanh_out = self.tanh(dilated_out[:, :dilated_out.shape[1]//2, :])
                sigmoid_out = self.sigmoid(dilated_out[:, dilated_out.shape[1]//2:, :])
                
                gated_out = tanh_out * sigmoid_out
                
                # Residual connection
                residual_out = self.residual_convs[layer_idx](gated_out)
                x = x + residual_out
                
                # Skip connection
                skip_out = self.skip_convs[layer_idx](gated_out)
                skip_connections.append(skip_out)
                
                layer_idx += 1
        
        # Sum all skip connections
        skip_sum = sum(skip_connections)  # [batch, skip_channels, seq_len]
        
        # Output layers
        output = self.relu(skip_sum)
        output = self.output_conv1(output)
        output = self.relu(output)
        output = self.output_conv2(output)
        
        # Apply softmax to get probabilities
        output = F.softmax(output, dim=1)  # [batch, num_classes, seq_len]
        
        return output

# Test the WaveNet architecture
print("Testing WaveNet architecture...")

batch_size = 2
seq_len = 100
input_channels = 2

# Create dummy input (character-level start/end probabilities)
dummy_input = torch.randn(batch_size, input_channels, seq_len)

# Initialize model
model = CharacterWaveNet(
    input_channels=input_channels,
    num_classes=2,
    num_blocks=2,  # Reduced for testing
    num_layers=4,   # Reduced for testing
    residual_channels=16,
    gate_channels=16,
    skip_channels=16
)

# Forward pass
with torch.no_grad():
    output = model(dummy_input)

print(f"Input shape: {dummy_input.shape}")
print(f"Output shape: {output.shape}")
print(f"Output sum (should be 1.0 per position): {output[0, :, 0].sum().item():.4f}")

Testing WaveNet architecture...


Input shape: torch.Size([2, 2, 100])
Output shape: torch.Size([2, 2, 100])
Output sum (should be 1.0 per position): 1.0000


In [ ]:
# Dataset for character-level training
class CharacterLevelDataset(Dataset):
    def __init__(self, texts, sentiments, selected_texts, roberta_model, tokenizer, device='cpu'):
        """
        Dataset for character-level WaveNet training.
        
        Args:
            texts: List of tweet texts
            sentiments: List of sentiment labels
            selected_texts: List of selected_text spans (targets)
            roberta_model: Trained RoBERTa model for generating token probabilities
            tokenizer: Tokenizer for text processing
        """
        self.texts = texts
        self.sentiments = sentiments
        self.selected_texts = selected_texts
        self.roberta_model = roberta_model
        self.tokenizer = tokenizer
        self.device = device
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        sentiment = self.sentiments[idx]
        selected_text = self.selected_texts[idx]
        
        # Get character-level start/end probabilities from RoBERTa
        token_probs = extract_token_probabilities(
            self.roberta_model, text, sentiment, self.tokenizer, self.device
        )
        
        # Convert to character-level probabilities
        char_start_probs = token_to_char_probabilities(
            token_probs['start_probs'],
            token_probs['offset_mapping'],
            len(text)
        )
        char_end_probs = token_to_char_probabilities(
            token_probs['end_probs'],
            token_probs['offset_mapping'],
            len(text)
        )
        
        # Create character-level targets (binary: 1 if character is start/end of selected_text)
        char_start_target = np.zeros(len(text))
        char_end_target = np.zeros(len(text))
        
        # Find selected_text in original text
        try:
            start_idx = text.index(selected_text)
            end_idx = start_idx + len(selected_text) - 1
            
            # Mark start and end positions
            if start_idx < len(text):
                char_start_target[start_idx] = 1.0
            if end_idx < len(text):
                char_end_target[end_idx] = 1.0
        except ValueError:
            # selected_text not found (shouldn't happen with clean data)
            pass
        
        # Stack start and end probabilities as input features
        char_features = np.stack([char_start_probs, char_end_probs], axis=0)  # [2, text_len]
        char_targets = np.stack([char_start_target, char_end_target], axis=0)  # [2, text_len]
        
        return {
            'features': torch.FloatTensor(char_features),
            'targets': torch.FloatTensor(char_targets),
            'text': text,
            'selected_text': selected_text
        }

# Test the dataset
print("Testing CharacterLevelDataset...")

# Use a small subset for testing
test_texts = train_df['text'].iloc[:10].tolist()
test_sentiments = train_df['sentiment'].iloc[:10].tolist()
test_selected_texts = train_df['selected_text'].iloc[:10].tolist()

# Create dataset
dataset = CharacterLevelDataset(
    texts=test_texts,
    sentiments=test_sentiments,
    selected_texts=test_selected_texts,
    roberta_model=model,
    tokenizer=tokenizer,
    device='cpu'
)

# Test getting an item
sample = dataset[0]
print(f"Sample features shape: {sample['features'].shape}")
print(f"Sample targets shape: {sample['targets'].shape}")
print(f"Text length: {len(sample['text'])}")
print(f"Selected text: '{sample['selected_text']}'")